In [1]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import Literal

In [2]:
import together
import chromadb
from langsmith.wrappers import wrap_openai
from langsmith import traceable

from tavily import TavilyClient
from typing_extensions import TypedDict
from openai import OpenAI
from together import Together
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from langgraph.graph import END, StateGraph, START, MessagesState

from dotenv.main import load_dotenv
load_dotenv()


In [3]:
load_dotenv()

True

In [4]:
LANGCHAIN_TRACING_V2 = True
LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"
LANGCHAIN_PROJECT = "my_test_project"

In [5]:
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

In [6]:
EMBEDDING_MODEL = "text-embedding-3-small"
CURR_LLM_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

In [7]:
together_client = wrap_openai(Together(api_key=TOGETHER_API_KEY))
together_client

In [8]:
@traceable
def llm_pipeline(user_input: str) -> str:
    result = together_client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}], model=CURR_LLM_MODEL
    )
    return result.choices[0].message.content

In [9]:
llm_pipeline("Hi there. What model are you?")

'I\'m an AI model known as Llama. Llama stands for "Large Language Model Meta AI."'

## CHROMA DB SET UP


In [10]:
embedding_function = OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY, model_name=EMBEDDING_MODEL
)
chroma_client = chromadb.Client()

In [13]:
collection = chroma_client.create_collection(
    name="marcus_collection", embedding_function=embedding_function
)

# collection = chroma_client.get_collection(
#     name="marcus_collection", embedding_function=embedding_function
# )

In [14]:
with open("marcus_quotes.txt", "r") as reader:
    marcus_list = reader.readlines()[:150]
ids_list = [f"id{i+1}" for i in range(len(marcus_list))]

In [16]:
collection.add(documents=marcus_list, ids=ids_list)

In [17]:
collection.query(
    query_texts=["I want to learn good morals and the government of my temper"],
    n_results=2,
)

{'ids': [['id1', 'id15']],
 'distances': [[0.6908390522003174, 1.1410863399505615]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['From my grandfather Verus I learned good morals and the government of my temper.\n',
   'From Maximus I learned self-government, and not to be led aside by anything; and cheerfulness in all circumstances, as well as in illness; and a just admixture in the moral character of sweetness and dignity, and to do what was set before me without complaining. I observed that everybody believed that he thought as he spoke, and that in all that he did he never had any bad intention; and he never showed amazement and surprise, and was never in a hurry, and never put off doing a thing, nor was perplexed nor dejected, nor did he ever laugh to disguise his vexation, nor, on the other hand, was he ever passionate or suspicious. He was accustomed to do acts of beneficence, and was ready to forgive, and was free from all falsehood; and he presented the a

In [18]:
collection.query(
    query_texts=[
        "Я хочу научиться хорошим моральным принципам и управлению моим темпераментом"
    ],
    n_results=1,
)["documents"]

[['From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.\n']]

In [19]:
collection.query(
    query_texts=[
        "Я хочу научиться хорошим моральным принципам и управлению моим темпераментом"
    ],
    n_results=1,
)

{'ids': [['id3']],
 'distances': [[1.3667497634887695]],
 'metadatas': [[None]],
 'embeddings': None,
 'documents': [['From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## TavilyClient setup


In [20]:
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [21]:
tavily_client.search("Who is Aomine Daiki?")

{'query': 'Who is Aomine Daiki?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Daiki Aomine - Kuroko no Basuke Wiki',
   'url': 'https://kurokonobasuke.fandom.com/wiki/Daiki_Aomine',
   'content': "Daiki Aomine (青峰 大輝 Aomine Daiki) was the ace player of the renowned Generation of Miracles and was the former partner/light of Tetsuya Kuroko in Teikō Junior High. After hearing a benched teammate insult Tetsuya Kuroko and his style of play during a match against Seirin High, Aomine did not hesitate to slam him against the lockers and warn that he, as someone who couldn't even earn a spot on the court, had no right to comment on the players' performances.[5] Aomine was similarly protective over Ryōta Kise following his match against Fukuda Sōgō Academy, as he chose to wait outside the venue to prevent Shōgo Haizaki from causing any further trouble for Kise or his team.[6]",
   'score': 0.9997063,
   'raw_content': None},
  {'title': "Kuroko's Basketb

In [22]:
tavily_client.search("Кто такой Борис Рыжий?", max_results=2)

{'query': 'Кто такой Борис Рыжий?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'История легендарного поэта Бориса Рыжего, который прожил всего 26 лет ...',
   'url': 'https://ngs.ru/text/culture/2024/09/09/74062475/',
   'content': 'История поэта Бориса Рыжего, который ушел из жизни в 26 лет и прославился на весь мир История поэта Бориса Рыжего, который ушел из жизни в 26 лет и прославился на весь мир По его словам, Борис Рыжий «был и жил целиком в поэзии, а это\xa0— смертельно». В 1991 году Рыжий пошел по стопам отца и поступил в Свердловский горный институт (на самом деле\xa0— потому что больше было некуда). Писатель Евгений Касимов рассказал E1.RU, что был не так близко знаком с Борисом: несколько раз они работали вместе, делали передачи на радио и ходили друг к другу в гости. Хамство, невежество и мракобесие: врач-педиатр — о том, что его бесит в пациентах Хамство, невежество и мракобесие: врач-педиатр — о том, что его бесит в пациентах',
 

## Prompts


In [23]:
router_prompt = """
    SYSTEM:
    You are an expert at routing a user question to a vectorstore or web search. 
    Vectorstore is consists of the quotes of Marcus Aurelius. If there is an emotion or need for
    an advicce for life situation or just a complain or an advice, 
    use the vectorstore. For all other questions use web-search
    Give a binary choice 'web_search' or 'vectorstore' based on the question. Return a string with a single word 'web_search' or 'vectorstore' and 
    no premable or explaination. 

    Example:
        Question: I want to learn good morals and the government of my temper.
        Answer: 'vectorstore'

    USER:
    """

In [24]:
stoic_prompt = """
    SYSTEM:
    You are a multilingual string merger. You have a user complain about something emotional in any language.
    You have a tool with a list of closest quotes 
    from Marcus Aurelius. Yus should pick up a most appropriate one quote and return it with the prefix 
    "That is what Marcus Avrelius said on this topic:" You must not rephrase the 
    quote or use any thoughts of yours. Just prefix and quote no metter what. Answer only in English.

    Example:
        Question: I want to learn good morals and the government of my temper.
        Quotes:["Be not disgusted, nor discouraged, nor dissatisfied,
          if thou dost not succeed in doing everything according to right principles.\n",
  "Examine men's ruling principles, even those of the wise, what kind of things they avoid, and what kind they pursue.\n"]

        Answer: "That is what Marcus Avrelius said on this topic: Be not disgusted, nor discouraged, nor dissatisfied,
          if thou dost not succeed in doing everything according to right principles

    USER:
    """

In [25]:
search_prompt = """
    SYSTEM:
    You are a multilingual search system. you have a qustion from user and a serach results. You need to the closest answer. 
    Answer always in English.

    USER:
    """

In [26]:
curr_question = "What can I do with my shitty life???"

llm_pipeline(f"{router_prompt} {curr_question}")

'vectorstore'

In [27]:
curr_question = "Who is Uzumaki Naruto?"

llm_pipeline(f"{router_prompt} {curr_question}")

'web_search'

## Agent python class

In [28]:
def get_search_results(query: str, max_results=2):
    return tavily_client.search(query, max_results=max_results)["results"]

In [29]:
def first_step_completion(user_query):
    return llm_pipeline(f"{router_prompt} {user_query}")

In [30]:
def second_step(first_step_completion, curr_question):
    if first_step_completion == "web_search":
        search_results = get_search_results(curr_question, max_results=2)
        return llm_pipeline( f"{search_prompt} {curr_question}, SEARCH_RESULTS: {search_results}" )

    elif first_step_completion == "vectorstore":
        store_search = collection.query(query_texts=[curr_question], n_results=2)[
            "documents"
        ]
        return llm_pipeline( f"{stoic_prompt} {curr_question}, QUOTES: {store_search}" )



In [31]:
def run(user_query):
    first_step_res = first_step_completion(user_query)
    return second_step(first_step_res, user_query)

In [32]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [33]:
run("I want to learn good morals and the government of my temper")

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smith.langchain.com/runs/batch', '{"detail":"int too big to convert"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smith.langchain.com/runs/batch', '{"detail":"int too big to convert"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smith.langchain.com/runs/batch', '{"detail":"int too big to convert"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smit

'That is what Marcus Aurelius said on this topic: From my grandfather Verus I learned good morals and the government of my temper.'

In [34]:
run("Some truths are not for you")

'That is what Marcus Aurelius said on this topic: Do not have such an opinion of things as he has who does thee wrong, or such as he wishes thee to have, but look at them as they are in truth.'

In [35]:
run("Shitty life! How to change it???")

'That is what Marcus Aurelius said on this topic: Hast thou seen those things? Look also at these. Do not disturb thyself. Make thyself all simplicity.'

In [36]:
run("Who is Uzumaki Naruto?")

'Based on the search results, the closest answer to the question "Who is Uzumaki Naruto?" is:\n\nNaruto Uzumaki is the titular protagonist of the manga Naruto, created by Masashi Kishimoto. He is a ninja from the fictional Hidden Leaf Village.'

In [37]:
run("Who is Aomine Daiki?")

'Based on the search results, Daiki Aomine is a character from the anime series "Kuroko no Basuke" (also known as "Kuroko\'s Basketball"). He is the ace player of the Generation of Miracles and was the former partner of Tetsuya Kuroko in Teikō Junior High. Aomine is known for his aggressive and unpredictable playing style, which is made even more powerful by Kuroko\'s misdirection. He is incredibly fast and versatile, with an uncanny ability to shoot from anywhere on the court.'

In [38]:
def tavily_search(query:str, max_results=2) -> dict:
    """
    Searches actual information in the Tavily system.

    Result example:
    'query': 'Who is Aomine Daiki?',
     max_results=2

  {
    'query': 'Who is Aomine Daiki?',
    'follow_up_questions': None,
    'answer': None,
    'images': [],
    'results': [
    {'title': "Aomine Daiki: The Phenomenal Basketball Star of Kuroko's Basketball",
    'url': 'https://dailyflares.com/aomine-daiki/',
    'content': "Aomine Daiki is a standout character in the popular anime and manga series Kuroko's Basketball. Known for his exceptional basketball skills and charismatic personality, Aomine has captured the hearts of fans worldwide. This article delves into the life and career of Aomine Daiki, exploring his role in Kuroko's Basketball, his basketball",
    'score': 0.9998233,
    'raw_content': None},
    {'title': 'Daiki Aomine - Kuroko no Basuke Wiki',
    'url': 'https://kurokonobasuke.fandom.com/wiki/Daiki_Aomine',
    'content': "Daiki Aomine (青峰 大輝 Aomine Daiki) was the ace player of the renowned Generation of Miracles and was the former partner/light of Tetsuya Kuroko in Teikō Junior High. After hearing a benched teammate insult Tetsuya Kuroko and his style of play during a match against Seirin High, Aomine did not hesitate to slam him against the lockers and warn that he, as someone who couldn't even earn a spot on the court, had no right to comment on the players' performances.[5] Aomine was similarly protective over Ryōta Kise following his match against Fukuda Sōgō Academy, as he chose to wait outside the venue to prevent Shōgo Haizaki from causing any further trouble for Kise or his team.[6]",
    'score': 0.9997063,
    'raw_content': None}
    ]
   }
    """
    return tavily_client.search(query, max_results=max_results)

In [39]:
def stoic_collection_search(query, n_results) -> dict:
    """
    Searches a stoic quote in a collection. Use when needed advice on an emotional or life-relational situation.

    {
    'ids': [['id1', 'id15']],
    'distances': [[0.6908390522003174, 1.1410863399505615]],
    'metadatas': [[None, None]],
    'embeddings': None,
    'documents': [['From my grandfather Verus I learned good morals and the government of my temper.\n',
    'From Maximus I learned self-government, and not to be led aside by anything; and cheerfulness in all circumstances, as well as in illness; and a just admixture in the moral character of sweetness and dignity, and to do what was set before me without complaining. I observed that everybody believed that he thought as he spoke, and that in all that he did he never had any bad intention; and he never showed amazement and surprise, and was never in a hurry, and never put off doing a thing, nor was perplexed nor dejected, nor did he ever laugh to disguise his vexation, nor, on the other hand, was he ever passionate or suspicious. He was accustomed to do acts of beneficence, and was ready to forgive, and was free from all falsehood; and he presented the appearance of a man who could not be diverted from right rather than of a man who had been improved. I observed, too, that no man could ever think that he was despised by Maximus, or ever venture to think himself a better man. He had also the art of being humorous in an agreeable way.\n']],
    'uris': None,
    'data': None,
    'included': ['metadatas', 'documents', 'distances']
    }
    """

    return collection.query(
        query_texts=[
            query
        ],
        n_results=n_results,
    ) # type: ignore

In [40]:
tools = [stoic_collection_search, tavily_search]

In [41]:
sys_msg = """
    SYSTEM:
    You are an expert at routing a user question to a vectorstore or web search. 
    Vectorstore is consists of the quotes of Marcus Aurelius. If there is an emotion or need for
    an advicce for life situation or just a complain or an advice, 
    use the vectorstore. For all other questions use web-search
    You should use stoic_collection_search or tavily_search based on the question. 

    USER:
    """


In [42]:
def assistant(state: MessagesState):
   return {"messages": [llm_pipeline([sys_msg] + state["messages"])]}

In [43]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.",
)

tools = [retriever_tool]

NameError: name 'retriever' is not defined

In [231]:
def node_one(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state']}

def route_node(user_query) -> Literal["web_search_node", "vectorestore_node"]:
    return llm_pipeline(f"{router_prompt} {user_query}") # type: ignore

In [232]:
def web_search_node(curr_question):
    search_results = get_search_results(curr_question, max_results=2)
    return llm_pipeline( f"{search_prompt} {curr_question}, SEARCH_RESULTS: {search_results}" )

def stoic_search_node(curr_question):
    store_search = collection.query(query_texts=[curr_question], n_results=2)[
        "documents"
    ]
    return llm_pipeline( f"{stoic_prompt} {curr_question}, QUOTES: {store_search}" )

In [233]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Build graph
builder.add_node("router_node", node_one)
builder.add_node("web_search_node", web_search_node)
builder.add_node("vectorestore_node", stoic_search_node)

# Logic
builder.add_edge(START, "router_node")
builder.add_conditional_edges("router_node", route_node)
builder.add_edge("web_search_node", END)
builder.add_edge("vectorestore_node", END)
# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"graph_state" : "Hi, this is Lance."})